In [1]:
!nvidia-smi

Sat May 27 13:54:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q git+https://github.com/UKPLab/sentence-transformers.git datasets

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.8 MB/s eta 0:0

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [2]:
from dataclasses import dataclass

@dataclass
class args:
    model_name = "LazarusNLP/simcse-indobert-base"
    dataset_name = "indonlp/indonlu"
    dataset_config = "smsa"
    train_split_name = "train"
    test_split_name = "test"
    text_column = "text"
    label_column = "label"

In [3]:
def preprocess(batch):
    sentences = batch[args.text_column]
    batch[args.text_column] = model.encode(sentences)
    return batch

In [4]:
model = SentenceTransformer(args.model_name)

In [5]:
dataset = load_dataset(args.dataset_name, args.dataset_config)
dataset = dataset.map(preprocess)
train_ds, test_ds = dataset[args.train_split_name], dataset[args.test_split_name]

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/11000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1260 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [6]:
classifier = LinearSVC()
classifier.fit(train_ds[args.text_column], train_ds[args.label_column])

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [7]:
predictions = classifier.predict(test_ds[args.text_column])

In [8]:
acc = accuracy_score(test_ds[args.label_column], predictions)
precision, recall, f1, _ = precision_recall_fscore_support(
    test_ds[args.label_column], predictions, average="macro"
)

In [9]:
results = {
    "accuracy": acc,
    "precision": precision,
    "recall": recall,
    "f1": f1,
}

In [10]:
print(results)

{'accuracy': 0.834, 'precision': 0.822428567566837, 'recall': 0.7682080990904522, 'f1': 0.782509656610376}
